In [1]:
import os
os.chdir('../../..')

In [2]:
import convokit
import pandas as pd
import numpy as np

In [3]:
CORPUS_DIR = "convokit/thread_generator/annotated-fake-trajectory"

In [48]:
from convokit import Corpus
from convokit import Clusterer
from sklearn.preprocessing import StandardScaler

In [5]:
corpus = Corpus(CORPUS_DIR)

## Hyperconv-20

In [6]:
corpus.print_summary_stats()

Number of Speakers: 200
Number of Utterances: 19980
Number of Conversations: 999


In [7]:
data = dict()
for convo in corpus.iter_conversations():
    data[convo.id] = convo.meta['hyperconvo-20']

In [8]:
df = pd.DataFrame(data).T

In [107]:
np.sum(np.sum(pd.isnull(df))) # checking for NaN vals

0

In [9]:
y_true = np.array([0]*333 + [1]*333 + [2]*333)

In [56]:
vals = StandardScaler().fit_transform(df.values)

## Hyperconvo-20, PCA (rank 9)

In [57]:
from sklearn.decomposition import PCA

In [58]:
pca = PCA(n_components=9)

In [59]:
pca_20_9 = pca.fit_transform(vals)

In [60]:
Clusterer.purity(pca_20_9, y_true)

0.7687687687687688

## Hyperconvo-20, PCA (rank 3)

In [61]:
pca_20_3 = PCA(n_components=3).fit_transform(vals)

In [62]:
Clusterer.purity(pca_20_3, y_true)

0.8518518518518519

## Hyperconvo-concat

In [86]:
for convo in corpus.iter_conversations():
    convo.meta['concat'] = dict()
    for idx in range(3, 20+1):
        d = convo.meta['hyperconvo-{}'.format(idx)].copy()
        convo.meta['concat'].update({k+"_"+str(idx): v for k, v in d.items()})

In [87]:
len(corpus.random_conversation().meta['concat'])

2520

In [88]:
2660 / 140

19.0

In [89]:
concat_data = dict()
for convo in corpus.iter_conversations():
    concat_data[convo.id] = convo.meta['concat']

In [127]:
concat_df = pd.DataFrame(concat_data).T

In [129]:
np.sum(np.sum(pd.isnull(concat_df))) # NaN values

12360

In [130]:
concat_vals = concat_df.values.astype('float64')

In [131]:
concat_vals[np.isnan(concat_vals)] = -1

In [132]:
concat_vals[np.isnan(concat_vals)]

array([], dtype=float64)

In [133]:
concat_vals = StandardScaler().fit_transform(concat_vals)

## PCA (rank 9)

In [139]:
pca_concat_9 = PCA(n_components=9).fit_transform(concat_vals)
Clusterer.purity(pca_concat_9, y_true)

0.9359359359359359

## PCA (rank 3)

In [140]:
pca_concat_3 = PCA(n_components=3).fit_transform(concat_vals)
Clusterer.purity(pca_concat_3, y_true)

0.9329329329329329

In [141]:
pca_concat_9

array([[ 32.79581832,  -3.67773664,  32.17694763, ...,  -4.78298362,
          0.3103629 ,   5.2829629 ],
       [ 38.95020812,  -9.38072444, -11.99405252, ...,  16.43580309,
        -19.56778948,   0.43145715],
       [ 43.79290604,  -9.99408821,  48.5498135 , ...,   9.79125929,
          9.5262141 ,   9.07122533],
       ...,
       [-25.26482742,  17.22759867,  10.37474599, ...,  19.95641962,
          7.00169012,  -4.68075883],
       [-22.12409456,  14.72896106,  17.70278678, ...,   3.25189752,
          1.03975168, -10.90308218],
       [-37.85369453, -11.91373247,   0.47802821, ...,  -6.33990889,
         -8.263612  ,   1.93409771]])

In [142]:
pca_concat_3

array([[ 32.79581832,  -3.67773666,  32.17694791],
       [ 38.95020812,  -9.38072443, -11.99405353],
       [ 43.79290604,  -9.99408824,  48.5498153 ],
       ...,
       [-25.26482742,  17.2275987 ,  10.37474589],
       [-22.12409456,  14.72896112,  17.70278578],
       [-37.85369453, -11.91373245,   0.47802845]])